In [ ]:
import os
import sys
import dlib
import cv2
import math
import numpy as np
from tqdm.auto import tqdm
import itertools

# location of the model (path of the model).
model_path = 'shape_predictor_68_face_landmarks.dat'

# catalogues (global values)
face_image_catalogue = './samples'
occ_catalogue = './occ'
result_catalogue = './results'

In [ ]:
# Prining/log functions
def print_run_failed(error_message):
    '''Print the failure message when the run fails.'''
    print('--- Run failed ---', file=sys.stderr)
    print(*error_message, sep='\n', file=sys.stderr)


def print_message(error_message):
    '''Print error messages encountered during the program execution.'''
    if not error_message:
        print('None errors during occlusion masking')
    else:
        print(f'Number of files skipped (error): {len(error_message)}')
        print('Errors detected during program run:', file=sys.stderr)
        print(*error_message, sep='\n', file=sys.stderr)
    return None


def print_result(error_message, generated_images_count):
    '''Print the results of the program execution.'''
    print('Occlusion(s) added: Mask, Cap, Glass, Sunglass')
    print(f'\nNumber of synthetic images generated: {generated_images_count}')
    print_message(error_message)
    return None


In [ ]:
def load_image_with_alpha(image_path):
    '''Load an image with an alpha channel, adding one if it doesn't exist.'''
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if isinstance(image, type(None)):
        return None
    if len(image.shape)<3: # Image content is not supported
        return None # skip this image, take the next one
    if image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    return image


def load_occlusion_images(occlusion_list):
    '''Load occlusion images from the occlusion list.'''
    face_mask_occ = load_image_with_alpha(occlusion_list[0])
    glass_occ = load_image_with_alpha(occlusion_list[1])
    sunglass_occ = load_image_with_alpha(occlusion_list[2])
    cap_occ = load_image_with_alpha(occlusion_list[3])
    return face_mask_occ, glass_occ, sunglass_occ, cap_occ


def check_catalogs_and_model(face_image_catalogue, occ_catalogue, model_path, error_message):
    '''Check existence of required catalogs and model.'''
    if not os.path.isdir(face_image_catalogue):
        error_message.append(
            f'Face image catalogue ({face_image_catalogue}) not found!')
    if not os.path.isdir(occ_catalogue):
        error_message.append(
            f'Occlusion image catalogue ({occ_catalogue}) not found!')
    if not os.path.isfile(model_path):
        error_message.append(f'Model file ({model_path}) not found!')
    return error_message


def check_and_get_occlusion_list(occ_catalogue, error_message):
    '''Check existence of required occlusions and return occlusion list'''
    required_occlusions = ['mask.png', 'glass.png', 'sunglass.png', 'cap.png']
    occlusion_list = []
    for occ in required_occlusions:
        if os.path.isfile(os.path.join(occ_catalogue, occ)):
            occlusion_list.append(os.path.join(occ_catalogue, occ))
        else:
            error_message.append(
                f'Occlusion file: {occ} does not exist in the occlusion image catalog')

    return occlusion_list, error_message


def get_image_list(face_image_catalogue):
    '''Get list of face images from the catalogue, and exclude synthetic images '''
    synthetic_file_ext = ['_glass.', '_mask.', '_cap.', '_sunglass.']
    image_list = []
    for image in os.listdir(face_image_catalogue):
        if image.endswith(('.jpg', '.png', '.jpeg')) and not any(ext in image for ext in synthetic_file_ext):
            image_list.append(os.path.join(face_image_catalogue, image))
    return image_list


def create_occlusion_combinations(occlusion_list):
    '''create combinations of occlusion to be applied '''
    occlusion_names = [os.path.basename(occ) for occ in occlusion_list]
    occlusion_combinations = [
        [occlusion_names[0], ''],
        [occlusion_names[1], occlusion_names[2], ''],
        [occlusion_names[3], '']
    ]
    return occlusion_combinations


In [ ]:
def calculate_maximal_area_rectangle_in_rotated_rectangle(width, height, angle):
    '''Calculate the largest possible axis-aligned rectangle in a rotated rectangle.
    Original code by coproc from Stack Overflow, but adapted for this project'''
    if width <= 0 or height <= 0:
      return 0,0

    width_is_longer = (width >= height)
    side_long, side_short = (width,height) if width_is_longer else (height,width)

    # since the solutions for angle, -angle and 180-angle are all the same,
    # if suffices to look at the first quadrant and the absolute values of sin,cos:
    sin_a, cos_a = abs(math.sin(angle)), abs(math.cos(angle))
    if side_short <= 2.*sin_a*cos_a*side_long or abs(sin_a-cos_a) < 1e-10:
      # half constrained case: two crop corners touch the longer side,
      #   the other two corners are on the mid-line parallel to the longer line
      x = 0.5*side_short
      return (x/sin_a,x/cos_a) if width_is_longer else (x/cos_a,x/sin_a)
    else:
      # fully constrained case: crop touches all 4 sides
      cos_2a = cos_a*cos_a - sin_a*sin_a
      return (width*cos_a - height*sin_a)/cos_2a, (height*cos_a - width*sin_a)/cos_2a

def rotate_image_by_angle(image, angle):
    '''Rotate the given image by "angle" in the range 0-360 degrees.'''
    height, width = image.shape[:2]
    center = (width/2, height/2)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1)

    return cv2.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))


def rotate_and_crop_image_by_angle(image, angle, crop):
    '''Rotate and optionally crop the image by the given angle in degrees. '''
    if not crop:
        return rotate_image_by_angle(image, angle)   # rotate only
    else:
        optimal_width, optimal_height = calculate_maximal_area_rectangle_in_rotated_rectangle(
           image.shape[1], image.shape[0],math.radians(angle))
        rotated = rotate_image_by_angle(image, angle)
        height, width, = rotated.shape[:2]
        y1 = height//2 - int(optimal_height/2)
        y2 = y1 + int(optimal_height)
        x1 = width//2 - int(optimal_width/2)
        x2 = x1 + int(optimal_width)

        return rotated[y1:y2, x1:x2]


def calculate_eye_center(face_landmarks):
    '''Calculate the center of each eye based on all 6 eye FaceLandmarks.'''
    r_eye_x = int(sum(face_landmarks.part(i).x for i in range(36, 42)) / 6)
    r_eye_y = int(sum(face_landmarks.part(i).y for i in range(36, 42)) / 6)
    l_eye_x = int(sum(face_landmarks.part(i).x for i in range(42, 48)) / 6)
    l_eye_y = int(sum(face_landmarks.part(i).y for i in range(42, 48)) / 6)

    return (r_eye_y, r_eye_x), (l_eye_y, l_eye_x)

def calculate_rotation_angle_for_eye_alignment(face_landmarks):
    '''Calculate the rotation angle of the face image for eye alignment.'''

    def get_angel (a, b, c):
        '''function calculates the angle between three 2-dimensional points a, b, and c. '''
        ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
        #return ang + 360 if ang < 0 else ang
        return ang

    r_eye_center, l_eye_center = calculate_eye_center(
        face_landmarks)
    l_eye_fixed = (l_eye_center[0], r_eye_center[1])
    return get_angel(r_eye_center, l_eye_center, l_eye_fixed)


def rotate_and_adjust_image_and_landmarks(face_image, angle, face_detector, landmark_predictor):
    '''Update the image and landmarks after rotating the image by the given angle.'''
    rotated_image = rotate_and_crop_image_by_angle(face_image, angle, True)
    image_RGB = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2RGB)

    # if face is not detected after cropping, rotate the image without cropping
    if len(face_detector(image_RGB, 0)) == 0:
        rotated_image = rotate_and_crop_image_by_angle(face_image, angle, False)
        image_RGB = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2RGB)

    face_image = rotated_image.copy()
    faces = face_detector(image_RGB, 0)
    face_rectangle_dlib = dlib.rectangle(int(faces[0].left()), int(
        faces[0].top()), int(faces[0].right()), int(faces[0].bottom()))
    detected_landmarks = landmark_predictor(image_RGB, face_rectangle_dlib)

    return face_image, detected_landmarks


def crop_cap_if_needed(face, cap, face_landmarks, increase_factor):
    '''Crop the resized cap if it extends beyond the face image boundaries.'''
    crop = False
    l_ear_x, r_ear_x = face_landmarks.part(0).x, face_landmarks.part(16).x
    l_eyebrow_y = face_landmarks.part(19).y
    r_eyebrow_y = face_landmarks.part(24).y
    nose_root_y = face_landmarks.part(27).y
    r_ear_x = min(r_ear_x, face.shape[1])
    l_ear_x = max(l_ear_x, 0)
    cap_lower_y_position = int((nose_root_y + (r_eyebrow_y + l_eyebrow_y)/2)/2)

    # Check top boundary
    if cap_lower_y_position < cap.shape[0]:
        start_pos_y = cap.shape[0] - cap_lower_y_position
        crop = True

    # Check left boundary
    x_left = int(l_ear_x - (cap.shape[1]*((increase_factor-1)/2)))
    if (x_left < 0):
        start_pos_x = -(x_left)
        crop = True
        x_left = 0
    else:
        start_pos_x = 0

    # Check right boundary
    x_right = int(r_ear_x + (cap.shape[1]*((increase_factor-1)/2)))
    if x_right > face.shape[1]:
        end_pos_x = cap.shape[1] - x_right + face.shape[1]
        crop = True
    else:
        end_pos_x = cap.shape[1]

    if crop:
        crop_cap = cap[start_pos_y:cap.shape[0],
                       start_pos_x:end_pos_x]
        cap = crop_cap.copy()
    return cap, x_left

In [ ]:
def add_occlusion_on_image(image, occlusion, x, y):
    '''Adding a occlusion on face image using alpha blending'''
    # Create "inv alpha' for alpha blending by subtracting norm. mask alpha from img.
    occlusion_alpha = occlusion[:, :, 3] / 255.0
    image_alpha = 1.0 - occlusion_alpha

    for channel in range(0, 3):
        image[y:y+occlusion.shape[0], x:x+occlusion.shape[1], channel] = (
            occlusion_alpha
            * occlusion[:, :, channel]
            + image_alpha
            * image[y:y+occlusion.shape[0], x:x+occlusion.shape[1], channel])
    return image

In [ ]:
def check_if_landmarks_within_image(face_landmarks, image, image_filename, error_message):
    '''Check if given landmarks are inside the image '''
    l_ear_x, l_ear_y = face_landmarks.part(0).x, face_landmarks.part(0).y
    r_ear_x, r_ear_y = face_landmarks.part(16).x, face_landmarks.part(16).y
    l_eye_y, r_eye_y = face_landmarks.part(19).y, face_landmarks.part(24).y
    chin_x, chin_y = face_landmarks.part(8).x, face_landmarks.part(8).y

    if l_ear_x < 0 or l_ear_y < 0 or l_ear_x > image.shape[1] or l_ear_y > image.shape[0]:
        error_message.append(
            f'Image {image_filename} face Landmarks left ear is out of bounds i.e. outside the face image)')
        return False, error_message
    if r_ear_x < 0 or r_ear_y < 0 or r_ear_x > image.shape[1] or r_ear_y > image.shape[0]:
        error_message.append(
            f'Image {image_filename} face Landmarks right ear is out of bounds i.e. outside the face image)')
        return False, error_message
    if l_eye_y < 0 or r_eye_y < 0:
        error_message.append(
            f'Image {image_filename} face Landmarks eyes are out of bounds(i.e. outside the face image)')
        return False, error_message
    if chin_x < 0 or chin_y < 0 or chin_x > image.shape[1]:
        error_message.append(
            f'Image {image_filename} face Landmarks chin is out of bounds i.e. outside the face image)')
        return False, error_message
    return True, error_message


def verify_frontal_face_orientation(face_landmarks, image_filename, error_message):
    '''Verify if face image photo is taken from front'''
    ratio = 4 # Ratio between ear and eye (left and right side)
    r_eye_center, l_eye_center = calculate_eye_center(face_landmarks)
    l_ear_x, r_ear_x = face_landmarks.part(0).x, face_landmarks.part(16).x
    distance_left, distance_right = r_eye_center[1] - \
        l_ear_x, r_ear_x - l_eye_center[1]

    if distance_left < distance_right and (distance_right / distance_left) > ratio:
        error_message.append(
            f'Image {image_filename} is not taken from front. Can not process the image')
        return False, error_message
    if distance_left > distance_right and (distance_left / distance_right) > ratio:
        error_message.append(
            f'Image {image_filename} is not taken from front. Can not process the image')
        return False, error_message
    return True, error_message

In [ ]:
def add_glasses(face, glass, face_landmarks):
    '''Add glass occlusion on a face image'''
    l_ear_x, r_ear_x = face_landmarks.part(0).x, face_landmarks.part(16).x
    r_eye_center, l_eye_center = calculate_eye_center(face_landmarks)
    r_ear_x = min(face.shape[1], r_ear_x)
    optimal_width = r_ear_x - l_ear_x
    scale = (optimal_width / glass.shape[1])
    resized_glass = cv2.resize(glass, None, fx=scale, fy=scale)

    # calculate x and y position on face where glasses should be placed
    y_pos = int(
        ((r_eye_center[0] + l_eye_center[0]) / 2) - (resized_glass.shape[0] / 3))
    x_pos = int(
        ((l_eye_center[1] + r_eye_center[1]) / 2) - (resized_glass.shape[1] / 2))

    if x_pos < 0: # Crop mask image in x-direction if it is out of bounds
        resized_glass = resized_glass[:, abs(x_pos):resized_glass.shape[1]]
        x_pos = 0

    # add occlusion on image with alpha blending and calculated position
    return add_occlusion_on_image(face, resized_glass, x_pos, y_pos)

In [ ]:
def add_cap (face, cap, face_landmarks):
    '''Add cap occlusion on an face image'''
    l_ear_x, r_ear_x = face_landmarks.part(0).x, face_landmarks.part(16).x
    l_eyebrow_y = face_landmarks.part(19).y
    r_eyebrow_y = face_landmarks.part(24).y
    nose_root_y = face_landmarks.part(27).y

    r_ear_x = min(r_ear_x, face.shape[1]) # fix r_ear_x if it is outside the image
    l_ear_x = max(l_ear_x, 0) # fix l_ear_x if it is outside the image

    increase_factor = 1.15 # 15 % width increase gives more realistic look
    optimal_width = int((r_ear_x - l_ear_x) * increase_factor)
    cap_scale = (optimal_width / cap.shape[1])

    # Resize the mask for maximum x-size inf face:
    resized_cap = cv2.resize(cap, None, fx=cap_scale, fy=cap_scale)
    cap_lower_y_position = int((nose_root_y + (l_eyebrow_y + r_eyebrow_y)/2)/2)

    resized_cap, x_pos = crop_cap_if_needed(
        face, resized_cap, face_landmarks, increase_factor)

    y_pos = cap_lower_y_position - resized_cap.shape[0]

    # add occlusion on image with alpha blending and calculated position
    return add_occlusion_on_image(face, resized_cap, x_pos, y_pos)


In [ ]:
def add_face_mask(face_img, face_mask, face_landmarks):
    '''Add cap occlusion on an face image'''
    l_ear_x, r_ear_x = face_landmarks.part(0).x, face_landmarks.part(16).x
    chin_y, nose_y = face_landmarks.part(8).y, face_landmarks.part(28).y

    # fix r_ear_x if it is outside the image
    r_ear_x = min(r_ear_x, face_img.shape[1]) # fix r_ear_x if it is outside the image
    l_ear_x = max(l_ear_x, 0) # fix l_ear_x if it is outside the image

    optimal_width = r_ear_x - l_ear_x
    optimal_height = chin_y - nose_y

    mask_scale_x = (optimal_width / face_mask.shape[1])
    mask_scale_y = (optimal_height / face_mask.shape[0])

    resized_face_mask = cv2.resize(face_mask, None, fx=mask_scale_x, fy=mask_scale_y)
    y_pos = chin_y - resized_face_mask.shape[0]
    if chin_y > face_img.shape[0]: # Crop in y direction if chin is outside image
        resized_face_mask = resized_face_mask[0:(face_img.shape[0] - y_pos), :]

    x_pos = l_ear_x # x-position for the mask

    # Mask the face:
    return add_occlusion_on_image(face_img, resized_face_mask, x_pos, y_pos)


In [ ]:
def create_occluded_face_images_and_export(image_list, occlusion_list):
    '''Add occlusions to a list of face images and save the results.'''
    error_message = []
    generated_images_count = 0

    # Initialize dlib frontal face detector and landmark predictor
    face_detector = dlib.get_frontal_face_detector()
    landmark_predictor = dlib.shape_predictor(model_path)

    face_mask_occ, glass_occ, sunglass_occ, cap_occ = load_occlusion_images(
        occlusion_list)

    '''For each face image filenames in the list '''
    for image in tqdm(image_list, file=sys.stdout, desc ="Images processed"):
        #tqdm.write(f"Downloading {image} files...")
        face_image = load_image_with_alpha(image)
        image_filename = os.path.basename(image)
        #print ("Processing: ", image_filename)
        # Validate images and skip if not valid
        if isinstance(face_image, type(None)):
            error_message.append(
                f'Image {image_filename} cannot be read by cv2 (missing file, improper permissions, unsupported or invalid format).')
            continue

        image_RGB = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
        faces = face_detector(image_RGB, 0)

        # Check for face detection issues and skip if needed
        if len(faces) == 0:
            error_message.append(
                f'Image {image_filename} does not contain a recognizable face')
            continue
        if len(faces) > 1:
            error_message.append(
                f'Image {image_filename} does contain more than one face')
            continue

        # Process face bounding box and landmarks
        face_rectangle_dlib = dlib.rectangle(int(faces[0].left()),
                                             int(faces[0].top()),
                                             int(faces[0].right()),
                                             int(faces[0].bottom()))
        detected_landmarks = landmark_predictor(image_RGB, face_rectangle_dlib)

        # Validate landmarks and face orientation, skip image if not valid
        landmarks_ok, error_message = check_if_landmarks_within_image(
            detected_landmarks, face_image, image_filename, error_message)
        face_from_front, error_message = verify_frontal_face_orientation(
            detected_landmarks, image_filename, error_message)
        if not landmarks_ok or not face_from_front:
            continue

        # Rotate the image if needed to align the face
        angle = calculate_rotation_angle_for_eye_alignment(detected_landmarks)
        if angle != 0:
            face_image, detected_landmarks = rotate_and_adjust_image_and_landmarks(
                face_image, angle, face_detector, landmark_predictor)

        # Apply occlusions and save the results
        combinations = create_occlusion_combinations(occlusion_list)
        for mask, glass, cap in itertools.product(combinations[0], combinations[1], combinations[2]):
            face_image_it = face_image.copy()
            #print(f'Combinations: {mask}, {glass}, {cap}')
            if not any([mask, glass, cap]):  # skip where no occlusion is 'selected'
                continue
            ext = ''
            if mask:
                ext += '_mask'
                face_image_it = add_face_mask(face_image_it, face_mask_occ,
                                           detected_landmarks)
            if glass:
                ext += '_glass' if glass == 'glass.png' else '_sunglass'
                occlusion = glass_occ if glass == 'glass.png' else sunglass_occ
                face_image_it = add_glasses(
                    face_image_it, occlusion, detected_landmarks)

            if cap:
                ext = ext + '_cap'
                face_image_it = add_cap(face_image_it, cap_occ, detected_landmarks)

            # Saving the result image with selected occlusions
            file_details = os.path.splitext(image_filename)
            new_name = file_details[0] + ext
            new_name_path = os.path.join(
                result_catalogue, new_name + file_details[1])
            cv2.imwrite(new_name_path, face_image_it)

            #print(f'angle: {angle}')
            # increment number of successful face image occlusion masking
            generated_images_count += 1

    # Print the result of the run
    print_result(error_message, generated_images_count)
    return None

In [ ]:
error_message = []

# Validate input paths and occlusion files
error_message = check_catalogs_and_model(
    face_image_catalogue,
    occ_catalogue,
    model_path,
    error_message
)
occlusion_list, error_message = check_and_get_occlusion_list(
    occ_catalogue, error_message)

if not os.path.isdir(result_catalogue):
    os.mkdir(result_catalogue)

# Get a list of face images from the provided catalog
image_list = get_image_list(face_image_catalogue)
if not image_list:
    error_message.append(
        f'Can not find any images (*.png, *.jpg, *.jpeg) in {face_image_catalogue}')

# Apply combinations of occlusions to all images and print results
if error_message:
    print_run_failed(error_message)
else:
    # Apply occlusions to all face images
    create_occluded_face_images_and_export(image_list, occlusion_list)